In [1]:
from fuvtds_base_class import FUVTDSBase
from fuvtds_base_class import FUVTDSMonitor

In [2]:
x = FUVTDSBase(PIDs='fuvtds_analysis_list.dat')

/Users/jhernandez/Desktop/fuvtds/monitor/fuvtds_base_class.py:1043: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  programs_df = pd.read_csv(PIDs, delim_whitespace=True)


  0%|          | 0/1766 [00:00<?, ?it/s]

35
35


34
34


34
34


35
35


34
34


34
34


